Google Cloud vision

In [1]:
! pip install --upgrade google-api-python-client
! pip install google-cloud-vision


     |████████████████████████████████| 6.5MB 5.3MB/s 
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
     |████████████████████████████████| 471kB 4.1MB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 512kB 6.2MB/s 
     |████████████████████████████████| 645kB 12.9MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
! cp ~/Downloads/sheet-contents-xxxx.json ~/aiffel/ocr_python/my_google_api_key.json

SyntaxError: ignored

In [ ]:
! export GOOGLE_APPLICATION_CREDENTIALS=$HOME/aiffel/ocr_python/my_google_api_key.json


In [ ]:
def detect_text(path):
   """Detects text in the file."""
   from google.cloud import vision
   import io
   client = vision.ImageAnnotatorClient()
​
   with io.open(path, 'rb') as image_file:
       content = image_file.read()
      
   image = vision.Image(content=content)
​
   response = client.text_detection(image=image)
   texts = response.text_annotations
   print('Texts:')
​
   for text in texts:
      print('\n"{}"'.format(text.description))
​
   vertices = (['({},{})'.format(vertex.x, vertex.y)
                for vertex in text.bounding_poly.vertices])
​
   print('bounds: {}'.format(','.join(vertices)))
​
   if response.error.message:
       raise Exception(
           '{}\nFor more info on error messages, check: '
           'https://cloud.google.com/apis/design/errors'.format(
               response.error.message))
# 다운받은 인증키 경로가 정확하게 지정되어 있어야 합니다.
!ls -l $GOOGLE_APPLICATION_CREDENTIALS
​
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] =  os.getenv('HOME')+'/aiffel/ocr_python/my_google_api_key.json'
​
# 입력 이미지 경로를 지정해 주세요.
# (예시) path = os.getenv('HOME')+'/aiffel/ocr_python/test_image.png'
path = # [[YOUR IMAGE FILE PATH]]  
​
# 위에서 정의한 OCR API 이용 함수를 호출해 봅시다.
detect_text(path)


KERAS

In [2]:
! pip list | grep tensorflow

tensorflow                    2.4.1         
tensorflow-datasets           4.0.1         
tensorflow-estimator          2.4.0         
tensorflow-gcs-config         2.4.0         
tensorflow-hub                0.11.0        
tensorflow-metadata           0.28.0        
tensorflow-probability        0.12.1        


In [3]:
! pip install keras-ocr

     |████████████████████████████████| 61kB 2.8MB/s 
     |████████████████████████████████| 9.5MB 5.8MB/s 
     |████████████████████████████████| 849kB 42.0MB/s 
     |████████████████████████████████| 133kB 45.2MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
  Created wheel for keras-ocr: filename=keras_ocr-0.8.6-cp37-none-any.whl size=42937 sha256=089d1561274dd6fd48f3f010e33d4a598a5739309e0b4e2f6b3b062878c27795
  Stored in directory: /root/.cache/pip/wheels/79/59/e6/79dd15333b911d51ea68d7e4c325e14f042eacc141201dca44
Successfully built keras-ocr


In [4]:
import matplotlib.pyplot as plt
import keras_ocr


In [6]:
# keras-ocr이 detector과 recognizer를 위한 모델을 자동으로 다운로드받게 됩니다.
pipeline = keras_ocr.pipeline.Pipeline()



Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5


In [8]:
# 테스트에 사용할 이미지 url을 모아 봅니다. 추가로 더 모아볼 수도 있습니다.
image_urls = [
 'https://source.unsplash.com/M7mu6jXlcns/640x460',
 'https://source.unsplash.com/6jsp4iHc8hI/640x460',
 'https://source.unsplash.com/98uYQ-KupiE',
 'https://source.unsplash.com/j9JoYpaJH3A',
 'https://source.unsplash.com/eBkEJ9cH5b4'
]
images = [ keras_ocr.tools.read(url) for url in image_urls]
prediction_groups = [pipeline.recognize([url]) for url in image_urls]


In [ ]:
# Plot the predictions
fig, axs = plt.subplots(nrows=len(images), figsize=(20, 20))
for idx, ax in enumerate(axs):
   keras_ocr.tools.drawAnnotations(image=images[idx],
                                   predictions=prediction_groups[idx][0], ax=ax)


Tesseract 

In [ ]:
! sudo apt install tesseract-ocr
! sudo apt install libtesseract-dev
! sudo apt install tesseract-ocr-kor


In [ ]:
! pip install pytesseract

In [ ]:
import os
import pytesseract
from PIL import Image
from pytesseract import Output
import matplotlib.pyplot as plt
​
# OCR Engine modes(–oem):
# 0 - Legacy engine only.
# 1 - Neural nets LSTM engine only.
# 2 - Legacy + LSTM engines.
# 3 - Default, based on what is available.
​
# Page segmentation modes(–psm):
# 0 - Orientation and script detection (OSD) only.
# 1 - Automatic page segmentation with OSD.
# 2 - Automatic page segmentation, but no OSD, or OCR.
# 3 - Fully automatic page segmentation, but no OSD. (Default)
# 4 - Assume a single column of text of variable sizes.
# 5 - Assume a single uniform block of vertically aligned text.
# 6 - Assume a single uniform block of text.
# 7 - Treat the image as a single text line.
# 8 - Treat the image as a single word.
# 9 - Treat the image as a single word in a circle.
# 10 - Treat the image as a single character.
# 11 - Sparse text. Find as much text as possible in no particular order.
# 12 - Sparse text with OSD.
# 13 - Raw line. Treat the image as a single text line, bypassing hacks that are Tesseract-specific.
​
def crop_word_regions(image_path='./images/sample.png', output_path='./output'):
   if not os.path.exists(output_path):
       os.mkdir(output_path)
   custom_oem_psm_config = r'--oem 3 --psm 3'
   image = Image.open(image_path)
​
   recognized_data = pytesseract.image_to_data(
       image, lang='eng',    # 한국어라면 lang='kor'
       config=custom_oem_psm_config,
       output_type=Output.DICT
   )
  
   top_level = max(recognized_data['level'])
   index = 0
   cropped_image_path_list = []
   for i in range(len(recognized_data['level'])):
       level = recognized_data['level'][i]
  
       if level == top_level:
           left = recognized_data['left'][i]
           top = recognized_data['top'][i]
           width = recognized_data['width'][i]
           height = recognized_data['height'][i]
          
           output_img_path = os.path.join(output_path, f"{str(index).zfill(4)}.png")
           print(output_img_path)
           cropped_image = image.crop((
               left,
               top,
               left+width,
               top+height
           ))
           cropped_image.save(output_img_path)
           cropped_image_path_list.append(output_img_path)
           index += 1
   return cropped_image_path_list
​
​
work_dir = os.getenv('HOME')+'/aiffel/ocr_python'
img_file_path = work_dir + '/test_image.png'   #테스트용 이미지 경로입니다. 본인이 선택한 파일명으로 바꿔주세요.


In [ ]:
def recognize_images(cropped_image_path_list):
   custom_oem_psm_config = r'--oem 3 --psm 7'
  
   for image_path in cropped_image_path_list:
       image = Image.open(image_path)
       recognized_data = pytesseract.image_to_string(
           image, lang='eng',    # 한국어라면 lang='kor'
           config=custom_oem_psm_config,
           output_type=Output.DICT
       )
       print(recognized_data['text'])
   print("Done")
​
# 위에서 준비한 문자 영역 파일들을 인식하여 얻어진 텍스트를 출력합니다.
recognize_images(cropped_image_path_list)
